In [68]:
import warnings
warnings.simplefilter("ignore")

from functools import partial
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset

import datetime, os

In [24]:
# load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# Import data

DATA_PATH = 'Reservoir_Project/Data'

In [3]:
basin_inflow_train = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_train.xlsx', index_col=0)
basin_inflow_validation = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_validation.xlsx', index_col=0)
basin_inflow_test = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_test.xlsx', index_col=0)

In [4]:
basin_inflow_train.head()

,INFLOW,ADR_PRECIP_ACC,ADR_PRECIP_INCR,ADR_TEMP_AVG,ADR_TEMP_MAX,ADR_TEMP_MIN,HYS_PRECIP_ACC,HYS_PRECIP_INCR,HYS_SNOW_DEPTH,HYS_SNOW_WATER_CONTENT,...,FRN_SNOW_DEPTH,FRN_SNOW_WATER_CONTENT,FRN_TEMP_AVG,FRN_TEMP_MAX,FRN_TEMP_MIN,PFH_PRECIP_ACC,PFH_PRECIP_INCR,PFH_TEMP_AVG,PFH_TEMP_MAX,PFH_TEMP_MIN
0,0.009407,-0.120687,0.184068,0.009505,-0.391558,0.331857,-0.064454,0.309236,-0.160654,-0.115369,...,-0.094061,-0.026854,-0.467328,-0.565277,-0.419978,-0.011883,0.153432,-0.125129,-0.568938,0.021703
1,0.009825,-0.115892,0.215971,-0.132046,-0.389896,0.092500,-0.059993,0.338807,-0.144326,-0.108889,...,-0.090099,-0.024826,-0.342022,-0.407445,-0.301333,-0.007179,0.223590,-0.117018,-0.685109,0.061987
2,0.003807,-0.113764,0.098568,-0.151271,-0.247423,0.010084,-0.055076,0.360112,-0.131400,-0.100130,...,-0.086392,-0.022915,-0.183639,-0.217192,-0.148991,-0.002509,0.221885,-0.174368,-0.534766,0.017436
3,-0.011969,-0.122304,-0.393446,-0.182306,-0.177232,-0.113069,-0.062791,-0.574891,-0.130901,-0.093803,...,-0.087422,-0.022983,-0.118318,-0.109488,-0.107866,-0.011679,-0.461838,-0.184934,-0.356396,-0.077629
4,-0.047851,-0.130268,-0.460592,-0.102898,-0.067602,-0.091750,-0.068891,-0.615032,-0.146738,-0.104076,...,-0.078444,-0.022474,-0.069900,-0.023878,-0.079156,-0.019216,-0.496717,-0.080716,-0.110395,-0.107840


### Data windowing

In [5]:
# TensorFlow utility class for producing data windows from time series data

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}
    
    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

In [6]:
"""
Window
- Given 60 days of history predict 30 days into the future. Why? A season is about 90 days in a CA WY
- Window size: 90
"""

window = WindowGenerator(input_width=60, label_width=1, shift=30,
                     train_df=basin_inflow_train, val_df=basin_inflow_validation, 
                     test_df=basin_inflow_test, label_columns=['INFLOW'])

window

In [7]:
# create a window of inputs and labels

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # set shapes after slicing
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [8]:
# create a dataset of sliding windows over a time series dataframe

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  # (input_window, label_window) pairs 
  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [9]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test

In [10]:
# each element is an (inputs, label) pair
window.train.element_spec

(TensorSpec(shape=(None, 60, 36), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [11]:
# example batch
for inputs, labels in window.train.take(1):
  print(f'Inputs shape (batch, time, features): {inputs.shape}')
  print(f'Labels shape (batch, time, features): {labels.shape}')

Inputs shape (batch, time, features): (32, 60, 36)
Labels shape (batch, time, features): (32, 1, 1)


### Modeling

#### Create baseline

In [12]:
# tensorflow baseline utility class for data windowing

class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [13]:
baseline = Baseline(label_index=window.column_indices['INFLOW'])

baseline.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
test_performance = {}
val_performance['Baseline'] = baseline.evaluate(window.val)
test_performance['Baseline'] = baseline.evaluate(window.test, verbose=0)

21/21 [==============================] - 0s 501us/step - loss: 0.1465 - mean_absolute_error: 0.1465


#### LSTM

In [61]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                optimizer=tf.keras.optimizers.legacy.Adam())

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, tensorboard_callback])
  return history

LSTM 1

In [21]:
"""
Inputs shape (batch, time, features): (32, 60, 36) - 32 batch size, 60 time steps, 36 features
"""

lstm_v1 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 36]),
    tf.keras.layers.Dense(units=1)
])

In [22]:
history_lstm_v1 = compile_and_fit(lstm_v1, window)

Epoch 1/10
79/79 [==============================] - 1s 9ms/step - loss: 0.0612 - val_loss: 0.1336
Epoch 2/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0410 - val_loss: 0.1348
Epoch 3/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0378 - val_loss: 0.1278
Epoch 4/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0354 - val_loss: 0.1458
Epoch 5/10
79/79 [==============================] - 1s 7ms/step - loss: 0.0337 - val_loss: 0.1453


In [25]:
%tensorboard --logdir logs

In [26]:
val_performance['LSTM1'] = lstm_v1.evaluate(window.val)
test_performance['LSTM1'] = lstm_v1.evaluate(window.test, verbose=0)

print('LSTM 1 Validation performance: ', val_performance['LSTM1'])
print('LSTM 1 Test performance: ', test_performance['LSTM1'])

21/21 [==============================] - 0s 2ms/step - loss: 0.1453
LSTM 1 Validation performance:  0.14531481266021729
LSTM 1 Test performance:  0.14204038679599762


LSTM 2

In [27]:
lstm_v2 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 36]),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [28]:
history_lstm_v2 = compile_and_fit(lstm_v2, window)

Epoch 1/10
79/79 [==============================] - 2s 16ms/step - loss: 0.0412 - val_loss: 0.1079
Epoch 2/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0305 - val_loss: 0.1262
Epoch 3/10
79/79 [==============================] - 1s 14ms/step - loss: 0.0272 - val_loss: 0.1304


In [29]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 45646), started 0:23:25 ago. (Use '!kill 45646' to kill it.)

In [30]:
val_performance['LSTM2'] = lstm_v2.evaluate(window.val)
test_performance['LSTM2'] = lstm_v2.evaluate(window.test, verbose=0)

print('LSTM 2 Validation performance: ', val_performance['LSTM2'])
print('LSTM 2 Test performance: ', test_performance['LSTM2'])

21/21 [==============================] - 0s 3ms/step - loss: 0.1304
LSTM 2 Validation performance:  0.13044767081737518
LSTM 2 Test performance:  0.06988170742988586


LSTM 3

In [94]:
RegularizedLSTM = partial(tf.keras.layers.LSTM,
                           dropout=0.01,
                           recurrent_dropout=0.01,
                           kernel_regularizer=tf.keras.regularizers.l2(0.05))

In [95]:
lstm_v3 = tf.keras.models.Sequential([
    RegularizedLSTM(32, return_sequences=True, input_shape=[None, 36]),
    RegularizedLSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [96]:
history_lstm_v3 = compile_and_fit(lstm_v3, window)

Epoch 1/10
79/79 [==============================] - 4s 33ms/step - loss: 3.0451 - val_loss: 1.5036
Epoch 2/10
79/79 [==============================] - 3s 34ms/step - loss: 0.7507 - val_loss: 0.4363
Epoch 3/10
79/79 [==============================] - 3s 34ms/step - loss: 0.1782 - val_loss: 0.1848
Epoch 4/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0614 - val_loss: 0.1371
Epoch 5/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0436 - val_loss: 0.1264
Epoch 6/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0377 - val_loss: 0.1441
Epoch 7/10
79/79 [==============================] - 3s 34ms/step - loss: 0.0377 - val_loss: 0.1413


In [89]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 45646), started 2:07:43 ago. (Use '!kill 45646' to kill it.)

In [97]:
val_performance['LSTM3'] = lstm_v3.evaluate(window.val)
test_performance['LSTM3'] = lstm_v3.evaluate(window.test, verbose=0)

print('LSTM 3 Validation performance: ', val_performance['LSTM3'])
print('LSTM 3 Test performance: ', test_performance['LSTM3'])

21/21 [==============================] - 0s 4ms/step - loss: 0.1413
LSTM 3 Validation performance:  0.14134171605110168
LSTM 3 Test performance:  0.06945193558931351
